# Relative correlation length

How to derive a relative correlation length based on the topography gradients

In [ ]:
using DIVAnd
using PyPlot
if VERSION >= v"0.7.0-beta.0"
    using Dates
    using Statistics
    using Random
else
    using Compat: @info, @warn, @debug
end
using Compat

In [ ]:
dx = dy = 0.1
lonr = 2.5:dx:12.
latr = 42.3:dy:44.6

In [ ]:
# https://b2drop.eudat.eu/s/ACcxUEZZi6a4ziR/download  gebco_30sec_4.nc
# https://b2drop.eudat.eu/s/x83MSV8yUF934bL/download  gebco_30sec_8.nc
# https://b2drop.eudat.eu/s/o0vinoQutAC7eb0/download  gebco_30sec_16.nc

bathname = "data/gebco_30sec_4.nc"

if !isfile(bathname)
    download("https://b2drop.eudat.eu/s/ACcxUEZZi6a4ziR/download",bathname)
else
    @info("Bathymetry file already downloaded")
end

bathisglobal = true

Read bathymetry and create mask

In [ ]:
bx,by,b = DIVAnd.load_bath(bathname,true,lonr,latr)
mask = b .< 0
b[b .< 0] .= 0;

In [ ]:
pcolor(bx,by,permutedims(b, [2,1]));  
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))
colorbar(orientation="horizontal")

In [ ]:
x,y = DIVAnd.ndgrid(bx,by);
pm,pn = DIVAnd.DIVAnd_metric(x,y)

L = 10_000 # meters

RL = DIVAnd.lengraddepth((pm,pn),b, L);

In [ ]:
pcolor(bx,by,permutedims(RL, [2,1])); 
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))
colorbar(orientation="horizontal")

In [ ]:
depthr = [0.,10.,20.]
length(depthr)
RL3D = repeat(RL,inner=(1,1,length(depthr)))
@show size(RL3D)

Parameter `len` for diva3d is then:

In [ ]:
len = (RL3D,RL3D,RL3D);

if the option `fitcorrlen` is true, internally diva3d will multiply the relative correlation length with the result of `fithorzcorr` and `fithvertcorr`. If fitting is not activated you have to multiply yourself the relative length scales by appropriate values